In [1]:
import torch


In [2]:
#testing device
torch.cuda.is_available()

True

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel
from PIL import Image
import json
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [4]:
#libs we used to accelerate the training process
!pip install accelerate -U
!pip install transformers[torch]
! pip install datasets
! pip install --upgrade tqdm
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
# Defining ResNet model for image feature extraction
resnet = models.resnet152(pretrained=True)
resnet.fc = nn.Identity()  # Removing final fully connected layer to extract the features
#resnet.eval()

C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
hierarchy = {

}

In [7]:
from langdetect import detect_langs, DetectorFactory
from googletrans import Translator

def detect_and_translate(text):
    try:
        # Detect the language of the text
        detected_langs = detect_langs(text)
        detected_lang = detected_langs[0].lang
        detected_prob = detected_langs[0].prob
        
        # If the detected language is not English and has high enough probability, translate it
        if detected_lang != 'en' and detected_prob > 0.5:
            translator = Translator()
            translated_text = translator.translate(text, src=detected_lang, dest='en').text
            return translated_text
        else:
            return text
    except:
        # If language detection fails or probability is too low, return original text
        return text

# Example usage
text_to_translate = "¡Hola! ¿Cómo estás?"
translated_text = detect_and_translate(text_to_translate)
print(translated_text)

Hello!How are you?


In [8]:
# Defining the dataset class
class MemeDataset_T(Dataset):
    def __init__(self, image_dir, annotation_file, tokenizer, label_mapping, transform=None):
        self.image_dir = image_dir
        with open(annotation_file, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.label_mapping = label_mapping  # Dictionary mapping label names to indices
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.image_dir, item['image'])
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)
        text = item['text']
        inputs = self.tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        # Convert labels to indices and then to a tensor
        label_indices = [self.label_mapping[label] for label in item['labels']]
        labels = torch.zeros(len(self.label_mapping))
        labels[label_indices] = 1  # One-hot encoding of labels

        return image, input_ids, attention_mask, labels


In [9]:
# Defining the dataset class
class MemeDataset_V(Dataset):
    def __init__(self, image_dir, annotation_file, tokenizer, label_mapping, transform=None):
        self.image_dir = image_dir
        with open(annotation_file, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.label_mapping = label_mapping  # Dictionary mapping label names to indices
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.image_dir, item['image'])
        image = Image.open(image_path).convert('RGB')
        image = self.transform(image)
        text = detect_and_translate(item['text'])
        inputs = self.tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)

        # Convert labels to indices and then to a tensor
        label_indices = [self.label_mapping[label] for label in item['labels']]
        labels = torch.zeros(len(self.label_mapping))
        labels[label_indices] = 1  # One-hot encoding of labels

        return image, input_ids, attention_mask, labels


In [10]:
class MemeClassifier(nn.Module):
    def __init__(self, text_model, num_labels):
        super(MemeClassifier, self).__init__()
        self.text_model = text_model
        self.image_model = resnet
        self.classifier = nn.Linear(self.text_model.config.hidden_size + 2048, num_labels)

    def forward(self, images, input_ids, attention_mask):
        with torch.no_grad():
            image_features = self.image_model(images)
        text_features = self.text_model(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        combined_features = torch.cat((text_features, image_features), dim=1)
        logits = self.classifier(combined_features)
        return logits


In [11]:
from sklearn.metrics import accuracy_score

def evaluate(model, data_loader, device, label_mapping):
    #print(model.eval())
    all_preds = []
    all_labels = []
    all_images = []  # Assuming you want to print images as well
    with torch.no_grad():
        for images, input_ids, attention_mask, labels in data_loader:
            #print("Labels is ",labels)
            images, input_ids, attention_mask, labels = images.to(device), input_ids.to(device), attention_mask.to(device), labels.to(device)
            logits = model(images, input_ids, attention_mask)
            #print("Logits is ",logits,"\n")
            predictions = torch.sigmoid(logits).round()  # Threshold at 0.5
            #print("Prediction is",predictions)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_images.extend(images)  # Append images or inputs if needed

    # Print predicted and actual labels
    correct = 0
    totalCount = 0
    for i in range(len(all_preds)):
        #print("All preds[i] is ",all_preds[i])
        predicted_labels = [label_mapping.get(j, 'Unknown') for j, label in enumerate(all_preds[i]) if label == 1]
        actual_labels = [label_mapping.get(j, 'Unknown') for j, label in enumerate(all_labels[i]) if label == 1]
        #actual_labels = [label_mapping[j] for j, label in enumerate(all_labels[i]) if label == 1]
        print("Predicted labels:", predicted_labels)
        print("Actual labels:", actual_labels)
        
        #correct = 0
        totalCount = totalCount + 1
        justCount = 0
    
        for j in actual_labels:
            if j in predicted_labels:
                justCount = justCount + 1

        if justCount >= len(actual_labels)/2:
            correct = correct + 1

    # Calculate manual accuracy checking the predicted labels correctly matching the true labels atleast half or more
    manual_accuracy = (correct / totalCount) * 100
    print("Manual Accuracy:", manual_accuracy)
    # Calculate hierarchical F1 score and accuracy
    h_f1 = hierarchical_f1_score(all_labels, all_preds, persuasion_hierarchy)
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Hierarchical F1 Score: {h_f1}")
    print(f"Accuracy: {accuracy}")
    #print("Accuracy manual is ",(correct/totalCount)*100) 

    return h_f1, accuracy


In [12]:
# Defining the hierarchy of persuasion techniques based on the provided image
persuasion_hierarchy = {
    'Persuasion': ['Ethos', 'Pathos', 'Logos'],
    'Ethos': ['Ad Hominem', 'Bandwagon', 'Appeal to Authority', 'Glittering Generalities', 'Transfer'],
    'Pathos': ['Appeal to Emotion', 'Exaggeration', 'Loaded Language', 'Flag Waving', 'Appeal to Fear', 'Transfer'],
    'Logos': ['Justification', 'Reasoning', 'Repetition', 'Intentional Vagueness'],
    'Ad Hominem': ['Name Calling', 'Doubt', 'Smears', 'Reductio ad Hitlerum', 'Whataboutism'],
    'Justification': ['Bandwagon', 'Appeal to Authority', 'Flag Waving', 'Appeal to Fear', 'Slogans'],
    'Reasoning': ['Distraction', 'Simplification'],
    'Distraction': ['Straw Man', 'Red Herring', 'Whataboutism'],
    'Simplification': ['Causal Oversimplification', 'Black & White Fallacy', 'Thought Terminating Cliche'],
}

In [13]:
# Function to get all ancestors of a technique in the hierarchy
def get_ancestors(technique, hierarchy):
    ancestors = []
    for parent, children in hierarchy.items():
        if technique in children:
            ancestors.append(parent)
            ancestors.extend(get_ancestors(parent, hierarchy))
    return ancestors

In [14]:
def hierarchical_f1_score(y_true, y_pred, hierarchy):
    """ Calculate the hierarchical F1 score considering the class hierarchy. """
    # Flattening the true and predicted labels to include their ancestors
    y_true_flat = [ancestor for label_set in y_true for label in label_set for ancestor in get_ancestors(label, hierarchy) + [label]]
    y_pred_flat = [ancestor for label_set in y_pred for label in label_set for ancestor in get_ancestors(label, hierarchy) + [label]]

    # Calculating F1 score
    f1 = f1_score(y_true_flat, y_pred_flat, average='micro')
    return f1

In [15]:
def create_label_mapping(annotation_file):
    all_labels = set()
    with open(annotation_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data:
            all_labels.update(item['labels'])
    return {label: i for i, label in enumerate(sorted(all_labels))}

In [16]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [17]:
def load_model(model, path):
    model.load_state_dict(torch.load(path))
    model.eval()  # Set the model to evaluation mode
    print("Model loaded and set to evaluation mode")

In [18]:
def load_annotation_data(annotation_file):
    with open(annotation_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [19]:
from tqdm import tqdm

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    text_model = BertModel.from_pretrained('bert-base-uncased')

    # Please provide your own paths to train our model if necessary
    image_dir = 'C:\\Users\\harih\\Downloads\\27\\train_images\\train_images'
    annotation_file = 'C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask2a\\preprocessed_train_v2.json'
    label_mapping = create_label_mapping(annotation_file)
    print("Label mapping is ", label_mapping)

    train_dataset = MemeDataset_T(image_dir, annotation_file, tokenizer, label_mapping)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

    print("Train Loader:", train_loader)

    # Kindly validate with json with labels 
    #labels are used only for checking purpose
    val_image_dir = 'C:\\Users\\harih\\Downloads\\27\\train_images\\dev_images'
    val_annotation_file = 'C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask2a\\dev_subtask2a_en.json'
    val_dataset = MemeDataset_V(val_image_dir, val_annotation_file, tokenizer, label_mapping)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

    print("Validation Loader:", val_loader)

    # Initialization of the model with the correct number of labels
    num_labels = 22  # Update this to reflect the correct number of unique labels
    model = MemeClassifier(BertModel.from_pretrained('bert-base-uncased'), num_labels).to(device)

    # Using optimizer and loss function here
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
    criterion = nn.BCEWithLogitsLoss()

    # Updated training loop with tqdm
    model.train()
    num_epochs = 15  
    for epoch in range(num_epochs):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
        for images, input_ids, attention_mask, labels in progress_bar:
            images, input_ids, attention_mask, labels = images.to(device), input_ids.to(device), attention_mask.to(device), labels.to(device)
            optimizer.zero_grad()
            logits = model(images, input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': total_loss / len(progress_bar)})  
        progress_bar.close()

    model_path = 'C:\\Users\\harih\\Downloads\\27\\meme_model_15f.pth'
    save_model(model, model_path)
    #evaluate(model, train_loader, device)
    #evaluate(model, val_loader, device)

    # Save the model after training
    
    save_model(model, model_path)

if __name__ == '__main__':
    main()


Device: cuda
Label mapping is  {'Appeal to (Strong) Emotions': 0, 'Appeal to authority': 1, 'Appeal to fear/prejudice': 2, 'Bandwagon': 3, 'Black-and-white Fallacy/Dictatorship': 4, 'Causal Oversimplification': 5, 'Doubt': 6, 'Exaggeration/Minimisation': 7, 'Flag-waving': 8, 'Glittering generalities (Virtue)': 9, 'Loaded Language': 10, "Misrepresentation of Someone's Position (Straw Man)": 11, 'Name calling/Labeling': 12, 'Obfuscation, Intentional vagueness, Confusion': 13, 'Presenting Irrelevant Data (Red Herring)': 14, 'Reductio ad hitlerum': 15, 'Repetition': 16, 'Slogans': 17, 'Smears': 18, 'Thought-terminating cliché': 19, 'Transfer': 20, 'Whataboutism': 21}
Train Loader: <torch.utils.data.dataloader.DataLoader object at 0x0000021AF758E1D0>
Validation Loader: <torch.utils.data.dataloader.DataLoader object at 0x0000021AFBE03B50>


Model saved to C:\Users\harih\Downloads\27\meme_model_15f.pth
Model saved to C:\Users\harih\Downloads\27\meme_model_15f.pth


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_model = BertModel.from_pretrained('bert-base-uncased')
image_dir = 'C:\\Users\\harih\\Downloads\\27\\train_images\\train_images'
annotation_file = 'C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask2a\\train.json'
label_mapping = create_label_mapping(annotation_file)
annotation_file = 'C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask2a\\train.json'
print(label_mapping)
train_dataset = MemeDataset_T(image_dir, annotation_file, tokenizer, label_mapping)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

{'Appeal to (Strong) Emotions': 0, 'Appeal to authority': 1, 'Appeal to fear/prejudice': 2, 'Bandwagon': 3, 'Black-and-white Fallacy/Dictatorship': 4, 'Causal Oversimplification': 5, 'Doubt': 6, 'Exaggeration/Minimisation': 7, 'Flag-waving': 8, 'Glittering generalities (Virtue)': 9, 'Loaded Language': 10, "Misrepresentation of Someone's Position (Straw Man)": 11, 'Name calling/Labeling': 12, 'Obfuscation, Intentional vagueness, Confusion': 13, 'Presenting Irrelevant Data (Red Herring)': 14, 'Reductio ad hitlerum': 15, 'Repetition': 16, 'Slogans': 17, 'Smears': 18, 'Thought-terminating cliché': 19, 'Transfer': 20, 'Whataboutism': 21}


In [21]:
reversed_label_mapping = {v: k for k, v in label_mapping.items()}
print(reversed_label_mapping)

{0: 'Appeal to (Strong) Emotions', 1: 'Appeal to authority', 2: 'Appeal to fear/prejudice', 3: 'Bandwagon', 4: 'Black-and-white Fallacy/Dictatorship', 5: 'Causal Oversimplification', 6: 'Doubt', 7: 'Exaggeration/Minimisation', 8: 'Flag-waving', 9: 'Glittering generalities (Virtue)', 10: 'Loaded Language', 11: "Misrepresentation of Someone's Position (Straw Man)", 12: 'Name calling/Labeling', 13: 'Obfuscation, Intentional vagueness, Confusion', 14: 'Presenting Irrelevant Data (Red Herring)', 15: 'Reductio ad hitlerum', 16: 'Repetition', 17: 'Slogans', 18: 'Smears', 19: 'Thought-terminating cliché', 20: 'Transfer', 21: 'Whataboutism'}


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Defining the model architecture
model = MemeClassifier(BertModel.from_pretrained('bert-base-uncased'), 22)

# Load the model's state_dict
model_state_dict = torch.load('C:\\Users\\harih\\Downloads\\27\\meme_model_15f.pth', map_location=device)

# Move the model parameters to the correct device
model.load_state_dict(model_state_dict)
model.to(device)

# Changing the model into evaluation mode
model.eval()

# Now Proceeding with evaluation
evaluate(model, train_loader, device, reversed_label_mapping)


In [23]:
val_image_dir = 'C:\\Users\\harih\\Downloads\\27\\dev_images\\dev_images'
val_annotation_file = 'C:\\Users\\harih\\Downloads\\27\\semeval2024_dev_release\\semeval2024_dev_release\\subtask2a\\dev_subtask2a_en.json'
val_dataset = MemeDataset_T(val_image_dir, val_annotation_file, tokenizer, label_mapping)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [24]:
print(val_loader)
evaluate(model, val_loader, device, reversed_label_mapping)

Predicted labels: []
Actual labels: ['Glittering generalities (Virtue)']
Predicted labels: ['Flag-waving', 'Loaded Language', 'Name calling/Labeling', 'Slogans', 'Smears']
Actual labels: ['Flag-waving', 'Loaded Language', 'Name calling/Labeling', 'Slogans', 'Smears']
Predicted labels: ['Loaded Language', 'Smears']
Actual labels: ['Loaded Language', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Glittering generalities (Virtue)', 'Smears']
Predicted labels: ['Black-and-white Fallacy/Dictatorship', 'Slogans']
Actual labels: ['Black-and-white Fallacy/Dictatorship']
Predicted labels: ['Smears']
Actual labels: ['Smears']
Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Flag-waving', 'Loaded Language', 'Name calling/Labeling']
Predicted labels: ['Smears']
Actual labels: ['Smears']
Predicted labels: ['Flag-waving', 'Loaded Language', 'Smears']
Actual labels: ['Flag-waving', 'Slogans']
Predicted labels: ['Smears']
Actual labels: ['Smears']
Predicted labels: ['Loa

(0.8943636363636364, 0.104)

In [25]:
#!pip install langdetect googletrans==4.0.0-rc1


In [26]:
#testing
text_to_translate = "Нещата започват да излизат извън\nконтрол!!"
translated_text = detect_and_translate(text_to_translate)
print(translated_text)


Things are starting to go beyond
Control !!


In [27]:
arabic_image_dir = 'C:\\Users\\harih\\Downloads\\multi_lang\\arabic_images\\test_images_arabic\\subtask2a'
arabic_annotation_file = 'C:\\Users\\harih\\Downloads\\multi_lang\\gold_multi\\test_subtask2a_ar.json'
arabic_dataset = MemeDataset_V(arabic_image_dir, arabic_annotation_file, tokenizer, label_mapping)
arabic_loader = DataLoader(arabic_dataset, batch_size=4, shuffle=False)

In [28]:
evaluate(model, arabic_loader, device, reversed_label_mapping)

Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Doubt', 'Smears']
Predicted labels: ['Loaded Language', 'Name calling/Labeling', 'Whataboutism']
Actual labels: ['Appeal to fear/prejudice', 'Doubt', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Exaggeration/Minimisation', 'Smears']
Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Causal Oversimplification', 'Exaggeration/Minimisation', 'Obfuscation, Intentional vagueness, Confusion']
Predicted labels: ['Loaded Language', 'Smears']
Actual labels: ['Appeal to fear/prejudice', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Appeal to fear/prejudice', 'Doubt', 'Exaggeration/Minimisation', 'Loaded Language', 'Name calling/Labeling', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Smears']
Predicted labels: ['Name calling/Labeling', 'Thought-terminating cliché']
Actual labels: ['Flag-waving', 'Name calling/Labeling', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['

(0.8871212121212121, 0.041666666666666664)

In [29]:
bulgarian_image_dir = 'C:\\Users\\harih\\Downloads\\multi_lang\\multi_images\\test_images\\subtask1_2a\\bulgarian'
bulgarian_annotation_file = 'C:\\Users\\harih\\Downloads\\multi_lang\\gold_multi\\test_subtask2a_bg.json'
bulgarian_dataset = MemeDataset_V(bulgarian_image_dir, bulgarian_annotation_file, tokenizer, label_mapping)
bulgarian_loader = DataLoader(arabic_dataset, batch_size=4, shuffle=False)

In [30]:
evaluate(model, bulgarian_loader, device, reversed_label_mapping)

Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Doubt', 'Smears']
Predicted labels: ['Loaded Language', 'Name calling/Labeling', 'Whataboutism']
Actual labels: ['Appeal to fear/prejudice', 'Doubt', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Exaggeration/Minimisation', 'Smears']
Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Causal Oversimplification', 'Exaggeration/Minimisation', 'Obfuscation, Intentional vagueness, Confusion']
Predicted labels: ['Loaded Language', 'Smears']
Actual labels: ['Appeal to fear/prejudice', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Appeal to fear/prejudice', 'Doubt', 'Exaggeration/Minimisation', 'Loaded Language', 'Name calling/Labeling', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['Smears']
Predicted labels: ['Name calling/Labeling', 'Thought-terminating cliché']
Actual labels: ['Flag-waving', 'Name calling/Labeling', 'Smears']
Predicted labels: ['Smears']
Actual labels: ['

(0.8871212121212121, 0.041666666666666664)

In [31]:
macedonian_image_dir = 'C:\\Users\\harih\\Downloads\\multi_lang\\multi_images\\test_images\\subtask1_2a\\north_macedonian'
macedonian_annotation_file = 'C:\\Users\\harih\\Downloads\\multi_lang\\gold_multi\\test_subtask2a_md.json'
macedonian_dataset = MemeDataset_V(macedonian_image_dir, macedonian_annotation_file, tokenizer, label_mapping)
macedonian_loader = DataLoader(macedonian_dataset, batch_size=4, shuffle=False)

In [32]:
evaluate(model, macedonian_loader, device, reversed_label_mapping)

Predicted labels: ['Appeal to fear/prejudice']
Actual labels: ['Loaded Language', 'Smears', 'Transfer']
Predicted labels: ['Loaded Language', 'Name calling/Labeling', 'Smears']
Actual labels: ['Appeal to fear/prejudice', 'Doubt', 'Loaded Language', "Misrepresentation of Someone's Position (Straw Man)", 'Slogans', 'Smears']
Predicted labels: ['Slogans', 'Smears']
Actual labels: ['Appeal to fear/prejudice', 'Flag-waving', 'Loaded Language', 'Name calling/Labeling', 'Smears', 'Transfer']
Predicted labels: ['Smears', 'Transfer']
Actual labels: ['Smears', 'Transfer']
Predicted labels: ['Smears']
Actual labels: ['Flag-waving', "Misrepresentation of Someone's Position (Straw Man)", 'Smears']
Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Exaggeration/Minimisation', 'Loaded Language', 'Smears', 'Transfer']
Predicted labels: ['Name calling/Labeling', 'Smears']
Actual labels: ['Loaded Language', 'Name calling/Labeling', 'Slogans', 'Smears']
Predicted labels: []
Actual lab

(0.8808353808353808, 0.05791505791505792)